<a href="https://colab.research.google.com/github/leohcar/P_IA025A_2022S1/blob/main/entrega_final/Proyecto_encoder_resnet50_smalldataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder

In [1]:
nome = 'Carlos Leonardo Ancasi Hinostroza'

print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


# Importação das bibliotecas

In [2]:
import collections
import itertools
import functools
import math
import random

import torch
import torchvision
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from torchvision.datasets import MNIST

In [3]:
# Check which GPU we are using
!nvidia-smi

Sat Jul 16 00:39:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print('GPU available:', device)

GPU available: cuda


In [5]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

#Dataset e dataloader

In [6]:
!mkdir ./data

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize([224,224]),
#      torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform = torchvision.transforms.Compose(
     [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
print(f'numero de imagenes {len(train_dataset)}')
img, target = train_dataset[0]
print(img.size())

numero de imagenes 50000
torch.Size([3, 32, 32])


In [8]:
# train_size = 40000
# val_size = 10000
# train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
train_size = 100
val_size = 10000
res_size = 50000 - train_size - val_size
train_dataset, val_dataset, _ = torch.utils.data.random_split(train_dataset, [train_size, val_size, res_size])

In [9]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 2
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([50, 3, 32, 32])
Valores mínimo e máximo dos pixels:  tensor(-1.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


#Bloco Básico da Resnet

In [10]:
from collections import OrderedDict
from torch import nn
from torch import Tensor


class MyBlock(nn.Module):

    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:
        super(MyBlock, self).__init__()
        # Escreva seu código aqui.
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # Escreva seu código aqui.
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [11]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:

        super(Bottleneck,self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=1, bias=False, padding=0)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.conv3 = nn.Conv2d(planes, planes*self.expansion, kernel_size=1, stride=1, bias=False, padding=0)
        self.bn3 = nn.BatchNorm2d(planes*self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)

        out += identity
        out = self.relu(out)
        return out


In [12]:
class CNN_resnet50(nn.Module):
    def __init__(self, image_channels, num_classes):

        super(CNN_resnet50,self).__init__()

        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #resnet layers
        self.layer1 = self.__make_layer(64, 64, 3,stride=1)
        self.layer2 = self.__make_layer(256, 128, 4,stride=2)
        self.layer3 = self.__make_layer(512, 256, 6,stride=2)
        self.layer4 = self.__make_layer(1024, 512, 3,stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.fc = nn.Linear(2048, num_classes)
      
    def __make_layer(self, in_channels, out_channels, blocks, stride):

        identity_downsample = None
        
        if stride != 1 or in_channels != 4*out_channels:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels, 4*out_channels, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(4*out_channels))
        layers = []
        layers.append(Bottleneck(in_channels, out_channels,identity_downsample=identity_downsample, stride=stride))
        
        for _ in range(1,blocks):
            layers.append(Bottleneck(out_channels*4, out_channels))

        layers.append(nn.Dropout(0.3))

        return nn.Sequential(*layers)
    
    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(x.shape[0], -1)
        out = self.fc(out)
        return out


In [13]:
model = CNN_resnet50(3,100).to(device)

In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 23712932


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
model_save_name = 'model_resnet50_smalldataset.pt'
path_modelo = F"/content/drive/My Drive/modelo_proyecto/entrega_03/pre_trained/{model_save_name}"

In [18]:
# Iniciar o treinamento
from tqdm import tqdm

n_epochs = 60
lr = 3.3e-4

img_channels = 3
num_classes = 100

model = CNN_resnet50(img_channels,num_classes).to(device)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

checkpoint = {
    'img_channels': img_channels,
    'num_classes': num_classes,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'epochs': [],
    'loss_train': [],
    'loss_val': [],
    'accuracy_history': [],
    }
torch.save(checkpoint, path_modelo)

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = []
accuracy_history = []
total_trained_samples = 0

with torch.no_grad():
    loss_accuracy = [list(validation_step(val_input_ids.to(device), val_target_ids.to(device)))
        for val_input_ids, val_target_ids in val_dataloader]
    
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()
    print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
    accuracy_history.append(mean_accuracy.item())

    checkpoint['accuracy_history'].append(mean_accuracy.item())
    checkpoint['loss_val'].append(mean_loss.item())
    torch.save(checkpoint, path_modelo)
    
for i in range(n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())
    
    checkpoint['model_state_dict'] = model.state_dict()
    checkpoint['optimizer_state_dict'] = optimizer.state_dict()

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0078000000000000005 Loss_Val: 4.605479357242584


Training: 100%|██████████| 2/2 [00:00<00:00, 12.89it/s]


Epoch: 0/59 Loss_Train: 5.203201770782471
Accuracy: 0.009800000000000001 Loss_Val: 4.607564692497253


Training: 100%|██████████| 2/2 [00:00<00:00, 17.31it/s]


Epoch: 1/59 Loss_Train: 4.347777843475342
Accuracy: 0.0103 Loss_Val: 4.619607737064362


...

Epoch: 57/59 Loss_Train: 0.7672209143638611
Accuracy: 0.0163 Loss_Val: 11.13502251625061


Training: 100%|██████████| 2/2 [00:00<00:00, 14.91it/s]


Epoch: 58/59 Loss_Train: 0.8942448496818542
Accuracy: 0.015700000000000002 Loss_Val: 11.081607427597046


Training: 100%|██████████| 2/2 [00:00<00:00, 18.67it/s]


Epoch: 59/59 Loss_Train: 0.7912945747375488
Accuracy: 0.015899999999999997 Loss_Val: 10.987197966575623


In [22]:
# continuar com o  treino
from tqdm import tqdm

n_epochs = 200
lr = 1e-4

checkpoint = torch.load(path_modelo)

model = CNN_resnet50(checkpoint['img_channels'],checkpoint['num_classes']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = checkpoint['loss_train']
accuracy_history = checkpoint['accuracy_history']
total_trained_samples = 0
epochs_init = checkpoint['epochs'][-1]+1

for s in zip(checkpoint['accuracy_history'],checkpoint['loss_val'],checkpoint['epochs'],checkpoint['loss_train']):
    print(f'Accuracy: {s[0]} Loss_Val: {s[1]}')
    print(f'')
    print(f'Epoch: {s[2]:d}/{n_epochs - 1:d} Loss_Train: {s[3]}')
print(f"Accuracy: { checkpoint['accuracy_history'][-1] } Loss_Val: {checkpoint['loss_val'][-1]}")

    
for i in range(epochs_init,n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())

    checkpoint['model_state_dict'] = model.state_dict()
    checkpoint['optimizer_state_dict'] = optimizer.state_dict()

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0078000000000000005 Loss_Val: 4.605479357242584

Epoch: 0/199 Loss_Train: 5.203201770782471
Accuracy: 0.009800000000000001 Loss_Val: 4.607564692497253

Epoch: 1/199 Loss_Train: 4.347777843475342
Accuracy: 0.0103 Loss_Val: 4.619607737064362

Epoch: 2/199 Loss_Train: 4.381622791290283
Accuracy: 0.009800000000000001 Loss_Val: 4.650605742931366

Epoch: 3/199 Loss_Train: 4.105149745941162
Accuracy: 0.009800000000000001 Loss_Val: 4.706207671165466

Epoch: 4/199 Loss_Train: 3.8122622966766357
Accuracy: 0.009800000000000001 Loss_Val: 4.783414611816406

Epoch: 5/199 Loss_Train: 4.317041873931885
Accuracy: 0.009800000000000001 Loss_Val: 4.873922731876373

Epoch: 6/199 Loss_Train: 3.9717276096343994
Accuracy: 0.009800000000000001 Loss_Val: 4.970412859916687

Epoch: 7/199 Loss_Train: 3.852762222290039
Accuracy: 0.0097 Loss_Val: 5.06861567735672

Epoch: 8/199 Loss_Train: 3.793790340423584
Accuracy: 0.009800000000000001 Loss_Val: 5.170141606330872

Epoch: 9/199 Loss_Train: 3.797158718109

Training: 100%|██████████| 2/2 [00:00<00:00,  6.00it/s]


Epoch: 160/199 Loss_Train: 0.19938720762729645
Accuracy: 0.023700000000000002 Loss_Val: 14.350396761894226


Training: 100%|██████████| 2/2 [00:00<00:00, 14.65it/s]


Epoch: 161/199 Loss_Train: 0.01886439323425293
Accuracy: 0.025099999999999997 Loss_Val: 14.429073567390441


...

Epoch: 197/199 Loss_Train: 0.27505406737327576
Accuracy: 0.022799999999999997 Loss_Val: 15.587585005760193


Training: 100%|██████████| 2/2 [00:00<00:00, 15.35it/s]


Epoch: 198/199 Loss_Train: 0.03509356826543808
Accuracy: 0.023799999999999998 Loss_Val: 15.31504289150238


Training: 100%|██████████| 2/2 [00:00<00:00, 14.14it/s]


Epoch: 199/199 Loss_Train: 0.024032514542341232
Accuracy: 0.024900000000000002 Loss_Val: 15.237982416152954


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=batch_size)

with torch.no_grad():
    loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
        for val_input_ids, val_target_ids in val_dataloader]
        
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()

print(f'TEST: Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')